In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import statsmodels.api as sm
from prophet import Prophet
from statsmodels.tsa.statespace.sarimax import SARIMAX
import xgboost as xgb
import plotly.graph_objects as go

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Function to process the bank statement Excel file
def process_bank_statement(file_path, skip_rows=21):
    """
    Processes the bank statement Excel file by skipping the first skip_rows rows.
    Extracts the date and withdrawal amount columns based on position.
    """
    try:
        # Read the Excel file, skipping the first skip_rows rows
        df = pd.read_excel(file_path, skiprows=skip_rows)
        
        # Debugging: Print the first few rows and columns
        print("Uploaded file preview:")
        print(df.head())
        print("Columns in the file:", df.columns.tolist())
        
        # Use column positions since names are obscured
        date_col = df.columns[0]  # First column (index 0) is the date
        withdrawal_col = df.columns[4]  # Fifth column (index 4) is the withdrawal amount
        
        # Select and rename columns
        processed_df = df[[date_col, withdrawal_col]].copy()
        processed_df.columns = ['date', 'amount']
        
        # Filter out rows with invalid dates (e.g., '********')
        processed_df = processed_df[processed_df['date'] != '********']
        
        # Convert date to datetime format
        processed_df['date'] = pd.to_datetime(processed_df['date'], format='%d/%m/%y', errors='coerce')
        
        # Drop rows with invalid dates (NaT)
        processed_df = processed_df.dropna(subset=['date'])
        
        # Convert withdrawal amount to numeric, handling any non-numeric values
        processed_df['amount'] = pd.to_numeric(processed_df['amount'], errors='coerce')
        
        # Filter for debit transactions only (non-zero withdrawal amounts)
        processed_df = processed_df[processed_df['amount'] > 0].copy()
        
        # Ensure amount is positive for analysis
        processed_df['amount'] = processed_df['amount'].abs()
        
        # Aggregate by date (in case there are multiple transactions per day)
        daily_spend = processed_df.groupby('date')['amount'].sum().reset_index()
        
        return daily_spend
    
    except Exception as e:
        print(f"Error processing file: {e}")
        return None

# Function to engineer features for time series forecasting
def engineer_features(df):
    """
    Creates time-based features for forecasting models.
    """
    df = df.copy()
    if 'date' in df.columns:
        df = df.set_index('date')
    
    # Resample to ensure regular time intervals (daily)
    df = df.resample('D').sum().fillna(0)
    
    # Add date-based features
    df = df.reset_index()
    df['dayofweek'] = df['date'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['day'] = df['date'].dt.day
    df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
    df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
    
    # Add lag features
    df['amount_lag7'] = df['amount'].shift(7)
    df['amount_lag14'] = df['amount'].shift(14)
    df['amount_lag30'] = df['amount'].shift(30)
    
    # Add rolling average features
    df['rolling_mean_7'] = df['amount'].rolling(window=7).mean()
    df['rolling_mean_14'] = df['amount'].rolling(window=14).mean()
    df['rolling_mean_30'] = df['amount'].rolling(window=30).mean()
    
    # Add rolling standard deviation (for volatility)
    df['rolling_std_7'] = df['amount'].rolling(window=7).std()
    df['rolling_std_30'] = df['amount'].rolling(window=30).std()
    
    # Weekly and monthly aggregations
    df['week_of_year'] = df['date'].dt.isocalendar().week
    df['day_of_month'] = df['date'].dt.day
    
    # Create cyclical features for day of week, month, etc.
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek']/7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek']/7)
    df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
    
    return df

# TimeSeriesForecaster class to train and evaluate models
class TimeSeriesForecaster:
    def __init__(self, data):
        self.original_data = data.copy()
        self.engineered_data = None
        self.best_model = None
        self.best_model_name = None
        self.best_score = float('inf')
        self.model_performances = {}
    
    def prepare_data(self):
        self.engineered_data = engineer_features(self.original_data)
        return self.engineered_data
    
    def train_test_split(self, test_size=0.2):
        if self.engineered_data is None:
            self.prepare_data()
        
        split_idx = int(len(self.engineered_data) * (1 - test_size))
        train = self.engineered_data.iloc[:split_idx].copy()
        test = self.engineered_data.iloc[split_idx:].copy()
        return train, test
    
    def evaluate_model(self, y_true, y_pred, model_name):
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-5))) * 100
        
        performance = {
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape
        }
        
        self.model_performances[model_name] = performance
        
        if mae < self.best_score:
            self.best_score = mae
            self.best_model_name = model_name
        
        return performance
    
    def train_prophet(self, forecast_periods=365):
        prophet_data = self.original_data.rename(columns={'date': 'ds', 'amount': 'y'})
        
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            seasonality_mode='multiplicative'
        )
        
        if len(prophet_data) > 60:
            model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
        
        model.fit(prophet_data)
        
        future = model.make_future_dataframe(periods=forecast_periods)
        forecast = model.predict(future)
        
        # Split the original data for test set alignment
        split_idx = int(len(self.original_data) * (1 - 0.2))  # Match test_size from train_test_split
        train_data = self.original_data.iloc[:split_idx]
        test_data = self.original_data.iloc[split_idx:]
        test_prophet = test_data.rename(columns={'date': 'ds', 'amount': 'y'})
        
        # Filter forecast to match test dates
        test_preds = forecast[forecast['ds'].isin(test_prophet['ds'])]['yhat'].values
        test_actuals = test_prophet['y'].values
        
        # Ensure lengths match before evaluation
        if len(test_preds) != len(test_actuals):
            raise ValueError(f"Length mismatch: test_preds={len(test_preds)}, test_actuals={len(test_actuals)}")
        
        performance = self.evaluate_model(test_actuals, test_preds, 'Prophet')
        
        return {
            'model': model,
            'forecast': forecast,
            'performance': performance
        }
    
    def train_sarima(self, forecast_periods=365):
        data = self.original_data.set_index('date')['amount']
        
        train, test = self.train_test_split()
        train_sarima = train.set_index('date')['amount']
        test_sarima = test.set_index('date')['amount']
        
        best_aic = float('inf')
        best_params = None
        
        p_values = [1, 2]
        d_values = [0, 1]
        q_values = [0, 1]
        P_values = [0, 1]
        D_values = [0, 1]
        Q_values = [0, 1]
        s_values = [7]  # Weekly seasonality
        
        for p in p_values:
            for d in d_values:
                for q in q_values:
                    for P in P_values:
                        for D in D_values:
                            for Q in Q_values:
                                for s in s_values:
                                    try:
                                        model = SARIMAX(
                                            train_sarima,
                                            order=(p, d, q),
                                            seasonal_order=(P, D, Q, s),
                                            enforce_stationarity=False,
                                            enforce_invertibility=False
                                        )
                                        result = model.fit(disp=False)
                                        aic = result.aic
                                        
                                        if aic < best_aic:
                                            best_aic = aic
                                            best_params = (p, d, q, P, D, Q, s)
                                    except:
                                        continue
        
        p, d, q, P, D, Q, s = best_params
        model = SARIMAX(
            train_sarima,
            order=(p, d, q),
            seasonal_order=(P, D, Q, s),
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        result = model.fit(disp=False)
        
        test_preds = result.predict(
            start=test.index[0],
            end=test.index[-1]
        )
        
        performance = self.evaluate_model(test_sarima.values, test_preds, 'SARIMA')
        
        forecast = result.get_forecast(steps=forecast_periods)
        forecast_values = forecast.predicted_mean
        forecast_ci = forecast.conf_int()
        
        return {
            'model': result,
            'forecast': {
                'mean': forecast_values,
                'lower': forecast_ci.iloc[:, 0],
                'upper': forecast_ci.iloc[:, 1]
            },
            'performance': performance,
            'params': best_params
        }
    
    def train_xgboost(self, forecast_periods=365):
        self.prepare_data()
        
        features = self.engineered_data.drop(['date', 'amount'], axis=1)
        target = self.engineered_data['amount']
        
        features = features.fillna(0)
        
        train, test = self.train_test_split()
        X_train = train.drop(['date', 'amount'], axis=1).fillna(0)
        y_train = train['amount']
        X_test = test.drop(['date', 'amount'], axis=1).fillna(0)
        y_test = test['amount']
        
        model = xgb.XGBRegressor(
            n_estimators=100,
            max_depth=5,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42
        )
        model.fit(X_train, y_train)
        
        test_preds = model.predict(X_test)
        
        performance = self.evaluate_model(y_test, test_preds, 'XGBoost')
        
        last_date = self.engineered_data['date'].iloc[-1]
        future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=forecast_periods)
        
        future_df = pd.DataFrame({'date': future_dates})
        future_df['amount'] = 0
        
        future_df['dayofweek'] = future_df['date'].dt.dayofweek
        future_df['month'] = future_df['date'].dt.month
        future_df['year'] = future_df['date'].dt.year
        future_df['day'] = future_df['date'].dt.day
        future_df['is_weekend'] = future_df['dayofweek'].isin([5, 6]).astype(int)
        future_df['is_month_start'] = future_df['date'].dt.is_month_start.astype(int)
        future_df['is_month_end'] = future_df['date'].dt.is_month_end.astype(int)
        
        future_df['dayofweek_sin'] = np.sin(2 * np.pi * future_df['dayofweek']/7)
        future_df['dayofweek_cos'] = np.cos(2 * np.pi * future_df['dayofweek']/7)
        future_df['month_sin'] = np.sin(2 * np.pi * future_df['month']/12)
        future_df['month_cos'] = np.cos(2 * np.pi * future_df['month']/12)
        
        future_df['week_of_year'] = future_df['date'].dt.isocalendar().week
        future_df['day_of_month'] = future_df['date'].dt.day
        
        full_data = self.engineered_data.copy()
        
        forecast_values = []
        
        for i in range(forecast_periods):
            current_df = pd.concat([full_data, future_df.iloc[:i]], axis=0)
            next_day = future_df.iloc[i:i+1].copy()
            
            for lag, days in [('amount_lag7', 7), ('amount_lag14', 14), ('amount_lag30', 30)]:
                lag_idx = len(current_df) - days
                if lag_idx >= 0:
                    next_day[lag] = current_df['amount'].iloc[lag_idx]
                else:
                    next_day[lag] = 0
            
            for window in [7, 14, 30]:
                roll_col = f'rolling_mean_{window}'
                roll_std_col = f'rolling_std_{window}'
                
                last_idx = len(current_df)
                start_idx = max(0, last_idx - window)
                
                if start_idx < last_idx:
                    recent_values = current_df['amount'].iloc[start_idx:last_idx]
                    next_day[roll_col] = recent_values.mean()
                    next_day[roll_std_col] = recent_values.std() if len(recent_values) > 1 else 0
                else:
                    next_day[roll_col] = 0
                    next_day[roll_std_col] = 0
            
            pred_cols = X_train.columns
            next_day_features = next_day[pred_cols]
            prediction = model.predict(next_day_features)[0]
            
            forecast_values.append(prediction)
            future_df.loc[future_df.index[i], 'amount'] = prediction
        
        return {
            'model': model,
            'forecast': {
                'dates': future_dates,
                'values': forecast_values
            },
            'performance': performance
        }
    
    def train_all_models(self, forecast_periods=365):
        prophet_results = self.train_prophet(forecast_periods)
        sarima_results = self.train_sarima(forecast_periods)
        xgboost_results = self.train_xgboost(forecast_periods)
        
        print("Model Performances:")
        for model_name, metrics in self.model_performances.items():
            print(f"{model_name} - MAE: {metrics['MAE']:.2f}, RMSE: {metrics['RMSE']:.2f}, MAPE: {metrics['MAPE']:.2f}%")
        
        print(f"\nBest model: {self.best_model_name} with MAE: {self.best_score:.2f}")
        
        return {
            'prophet': prophet_results,
            'sarima': sarima_results,
            'xgboost': xgboost_results,
            'best_model': self.best_model_name
        }

# Main function to run the forecasting
def main():
    # Provide the file path to your Excel file
    file_path = "ba_st.xlsx"  # Replace with your file path
    
    # Process the bank statement
    processed_df = process_bank_statement(file_path)
    
    if processed_df is None:
        print("Error processing file. Please check the format.")
        return
    
    # Initialize the forecaster
    forecaster = TimeSeriesForecaster(processed_df)
    
    # Train all models and generate forecasts
    results = forecaster.train_all_models(forecast_periods=365)
    
    # Visualize the best model's forecast
    best_model_name = results['best_model']
    best_forecast = results[best_model_name.lower()]['forecast']
    
    if best_model_name == 'Prophet':
        df = best_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=df['ds'],
            y=df['yhat'],
            mode='lines',
            name='Forecast',
            line=dict(color='rgba(220, 53, 69, 0.8)', width=2),
        ))
        fig.add_trace(go.Scatter(
            x=df['ds'].tolist() + df['ds'].tolist()[::-1],
            y=df['yhat_upper'].tolist() + df['yhat_lower'].tolist()[::-1],
            fill='toself',
            fillcolor='rgba(220, 53, 69, 0.2)',
            line=dict(color='rgba(255, 255, 255, 0)'),
            name='95% Confidence Interval',
            hoverinfo='skip'
        ))
        fig.update_layout(
            title='Spending Forecast (Prophet)',
            xaxis_title='Date',
            yaxis_title='Amount ($)',
            template='plotly_dark'
        )
    else:
        # Handle different forecast structures for SARIMA and XGBoost
        if best_model_name == 'SARIMA':
            df = pd.DataFrame({
                'date': best_forecast['mean'].index,
                'mean': best_forecast['mean'],
                'lower': best_forecast['lower'],
                'upper': best_forecast['upper']
            })
            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=df['date'],
                y=df['mean'],
                mode='lines',
                name='Forecast',
                line=dict(color='rgba(220, 53, 69, 0.8)', width=2),
            ))
            fig.add_trace(go.Scatter(
                x=df['date'].tolist() + df['date'].tolist()[::-1],
                y=df['upper'].tolist() + df['lower'].tolist()[::-1],
                fill='toself',
                fillcolor='rgba(220, 53, 69, 0.2)',
                line=dict(color='rgba(255, 255, 255, 0)'),
                name='95% Confidence Interval',
                hoverinfo='skip'
            ))
        elif best_model_name == 'XGBoost':
            df = pd.DataFrame({
                'date': best_forecast['dates'],
                'mean': best_forecast['values']
            })
            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=df['date'],
                y=df['mean'],
                mode='lines',
                name='Forecast',
                line=dict(color='rgba(220, 53, 69, 0.8)', width=2),
            ))
            # XGBoost doesn't provide confidence intervals in this implementation
            print("Note: XGBoost forecast does not include confidence intervals.")
        
        fig.update_layout(
            title=f'Spending Forecast ({best_model_name})',
            xaxis_title='Date',
            yaxis_title='Amount ($)',
            template='plotly_dark'
        )
    
    fig.show()

# Run the main function
if __name__ == '__main__':
    main()

02:56:09 - cmdstanpy - INFO - Chain [1] start processing


Uploaded file preview:
       Date                                          Narration  \
0  02/02/22        NWD-512967XXXXXX1000-07074019-MUMBAI SUBURB   
1  02/02/22           ACH D- YES BANK RETAIL ASSE-ALN000100902   
2  02/02/22  IMPS-203318120357-ZAHID MIR-CNRB-XXXXXXXXX5442...   
3  02/02/22  IMPS-203318120623-TALAT S SHAIKH-ICIC-XXXXXXXX...   
4  03/02/22               NWD-512967XXXXXX1000-RD117601-MUMBAI   

       Chq./Ref.No.  Value Dt  Withdrawal Amt.  Deposit Amt.  Closing Balance  
0  0000203308014831  02/02/22           7000.0           NaN        272658.16  
1  0000008593760278  02/02/22          16183.0           NaN        256475.16  
2  0000203318120357  02/02/22           1000.0           NaN        255475.16  
3  0000203318120623  02/02/22          10000.0           NaN        245475.16  
4  0000203415002355  03/02/22           9000.0           NaN        236475.16  
Columns in the file: ['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt', 'Withdrawal Amt.', 'Deposit A

02:56:09 - cmdstanpy - INFO - Chain [1] done processing


Model Performances:
Prophet - MAE: 15753.12, RMSE: 36770.03, MAPE: 384.64%
SARIMA - MAE: 19853.99, RMSE: 31782.42, MAPE: 26561742524.06%
XGBoost - MAE: 15355.77, RMSE: 28276.80, MAPE: 23438469858.04%

Best model: XGBoost with MAE: 15355.77
Note: XGBoost forecast does not include confidence intervals.
